# Setup environment

In [21]:
# Essentials
import os
import numpy as np
import importlib

# PyTorch
import torch
from torch.utils.data import DataLoader

# Utils
from utils import get_dict, get_files, check_corrupted_imgs, perturb_image, save_image, make_dirs, get_model
from utils import CustomTransforms, My_data, FocalLoss

# OnePixelAttack
import OnePixelAttack

importlib.reload(OnePixelAttack)

<module 'OnePixelAttack' from 'c:\\Users\\juliu\\OneDrive - Universiteit Twente\\2023-2024\\2. Machine Learning II\\Project\\fixed_code\\OnePixelAttack.py'>

## Setup Cuda

In [2]:
# Set a higher max split size to avoid memory problems
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [3]:
global device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(torch.cuda.memory_summary(device=None, abbreviated=False))
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False

cpu


## Get the data

In [4]:
# Load train and test files that are used for the model.
train_dict = get_dict("train.txt")
print("done")
test_dict = get_dict("test.txt")
print("done")

print(check_corrupted_imgs(train_dict, test_dict))

print("copying completed")

Opening Train.txt
done
Opening Train.txt
done
True
copying completed


In [5]:
train_files = get_files("./dataset/train/**/**/*.png")

test_files = get_files("./dataset/test/**/**/*.png")

# print(train_files)
print(test_files)

['./dataset/test\\A\\100\\SOB_B_A-14-22549AB-100-001.png', './dataset/test\\A\\100\\SOB_B_A-14-22549AB-100-010.png', './dataset/test\\A\\100\\SOB_B_A-14-22549AB-100-016.png', './dataset/test\\A\\100\\SOB_B_A-14-22549AB-100-017.png', './dataset/test\\A\\100\\SOB_B_A-14-22549AB-100-018.png', './dataset/test\\A\\100\\SOB_B_A-14-22549AB-100-026.png', './dataset/test\\A\\100\\SOB_B_A-14-22549AB-100-028.png', './dataset/test\\A\\100\\SOB_B_A-14-22549CD-100-022.png', './dataset/test\\A\\100\\SOB_B_A-14-22549G-100-024.png', './dataset/test\\A\\100\\SOB_B_A-14-29960CD-100-005.png', './dataset/test\\A\\100\\SOB_B_A-14-29960CD-100-007.png', './dataset/test\\A\\200\\SOB_B_A-14-22549AB-200-004.png', './dataset/test\\A\\200\\SOB_B_A-14-22549AB-200-006.png', './dataset/test\\A\\200\\SOB_B_A-14-22549AB-200-015.png', './dataset/test\\A\\200\\SOB_B_A-14-22549AB-200-019.png', './dataset/test\\A\\200\\SOB_B_A-14-22549CD-200-024.png', './dataset/test\\A\\200\\SOB_B_A-14-22549CD-200-027.png', './dataset/tes

## Setup Transformers

In [6]:
custom_transforms = CustomTransforms(test_files)
resize_transform = custom_transforms.get_transform("resize_tensor")
test_transform = custom_transforms.get_transform("test")

# Simple OnePixel Attack 

In [7]:
original = My_data([test_files[0]])[0][0]

p_tensor = np.copy(original)

for i in range(100):
    perturbation = [i, i] + list(np.random.choice(range(256), size=3))

    p_tensor = perturb_image(perturbation, p_tensor)

save_image(p_tensor, "perturbedimagerandom.png")


In [8]:
tensor_list = [original, p_tensor]

org = resize_transform(image=original)
org_im = org['image']

org_norm = test_transform(image=original)
org_norm_im = org_norm['image']

save_image(org_im, "org.png")
save_image(org_norm_im, "org_norm.png")

In [9]:
per = resize_transform(image=p_tensor)
per_im = per['image']

per_norm = test_transform(image=p_tensor)
per_norm_im = per_norm['image']

save_image(per_im, "per.png")
save_image(per_norm_im, "per_norm.png")

In [ ]:
# Deze wordt nu niet gecallt om tijd te besparen tijdens run all (nodig na veranderen geimporteerd bestand)
def simple_one_pixel_attack():
    make_dirs("simple_one_pixel")
    orgs = My_data(test_files)
    for i in range(orgs.__len__()):
        p_tensor = np.copy(orgs.__getitem__(i)[0])
        for j in range(200):
            random_x = np.random.choice(range(1, 698))
            random_y = np.random.choice(range(1, 398))

            random_xs = [random_x - 1, random_x, random_x + 1]
            random_ys = [random_y - 1, random_y, random_y + 1]

            for x in random_xs:
                for y in random_ys:
                    perturbation = [y, x] + list(np.random.choice(range(256), size=3))
                    p_tensor = perturb_image(perturbation, p_tensor)
                    
        save_image(p_tensor, os.path.join(os.getcwd(), "dataset", "simple_one_pixel", orgs.__getclass__(i), orgs.__getzoom__(i), orgs.__getname__(i)))


In [12]:
org_norm = My_data(test_files, transforms=test_transform)
perturb_data = My_data(get_files("./dataset/simple_one_pixel/**/**/*.png"), transforms=test_transform)

org_dataloader = DataLoader(org_norm)
pertrubed_dataloader = DataLoader(perturb_data)

## Test performance of Simple OnePixel Attack

In [17]:
model = get_model(device, test_dict)

model.eval()
correct_org = 0
correct_pert = 0
confs_org = []
confs_pert = []
with torch.no_grad():
    print("Testing network without attacks...")
    for i, (inputs, labels) in enumerate(org_dataloader):
        inputs = inputs.to(device)
        labels = labels.float()
        labels = labels.to(device)
        outputs = model(inputs)
        confs_org.append(outputs)
        
        if torch.argmax(outputs) == torch.argmax(labels):
            correct_org += 1
    print(f"Accuracy of network without attack: {correct_org/len(test_files)}")
# ######################## Run TinySwin without attacks ########################
    print("Testing network with OnePixel attack...")
    for i, (inputs, labels) in enumerate(pertrubed_dataloader):
        inputs = inputs.to(device)
        labels = labels.float()
        labels = labels.to(device)
        outputs = model(inputs)
        confs_pert.append(outputs)
        
        if torch.argmax(outputs) == torch.argmax(labels):
            correct_pert += 1
    print(f"Accuracy of network with OnePixel attack: {correct_pert/len(test_files)}")

    # Compare the performance in confidence outputs
    print(sum([torch.max(confs_org[i]) > torch.max(confs_pert[i]) for i in range(len(test_files))]), len(test_files))

Testing network without attacks...
Accuracy of network without attack: 0.9665379665379665
Testing network with OnePixel attack...
Accuracy of network with OnePixel attack: 0.9678249678249679
tensor(452) 777


In [18]:
# Show all differences in predictions
for i in range(len(test_files)):
    if torch.argmax(confs_org[i]) != torch.argmax(confs_pert[i]):
        print("Different predictions")
        print(confs_org[i])
        print(confs_pert[i])
        print(test_files[i])

Different predictions
tensor([[ -8.9458, -11.2881, -11.0125, -13.2905,  12.1696,  12.5147, -12.2090,
         -11.1257]])
tensor([[ -8.9580, -11.0795, -10.6683, -13.3457,  12.7220,  12.2752, -12.0030,
         -11.1058]])
./dataset/test\DC\400\SOB_M_DC-14-13412-400-021.png
Different predictions
tensor([[-7.8177, -8.9137, -9.0588, -9.4414,  0.9318,  2.4278, -3.8109, -9.0984]])
tensor([[-7.7231, -8.6616, -8.5861, -9.2060,  1.5346,  1.1586, -3.3869, -8.9427]])
./dataset/test\MC\400\SOB_M_MC-14-16456-400-004.png


# Advanced OnePixel Attack

In [22]:
success = 0
for i, (image, label) in enumerate(org_dataloader):
    success += OnePixelAttack.attack(i, model, device, image, label, pixel_count=1, maxiter=50, popsize=15)
    
print(success)

5
[  3.82848831 144.07396674   4.52771629   4.85545761   8.76782544]
Original tensor(10.5070, grad_fn=<SelectBackward0>)
Perturbed tensor(10.4920, grad_fn=<SelectBackward0>)
5
[105.59902654  72.94206342   9.16306156   8.09628307  -7.12113913]
Checking original images
Original tensor(10.5070, grad_fn=<SelectBackward0>)
Perturbed tensor(10.5247, grad_fn=<SelectBackward0>)
5
[204.48901731 176.58970921  -6.20861485  -7.22606797  -0.51296918]
Checking original images
Original tensor(10.5070, grad_fn=<SelectBackward0>)
Perturbed tensor(10.5117, grad_fn=<SelectBackward0>)
5
[132.1854769  168.22842466  -9.56156602  -3.0491135    2.53340362]
Checking original images
Original tensor(10.5070, grad_fn=<SelectBackward0>)
Perturbed tensor(10.5056, grad_fn=<SelectBackward0>)
5
[ 18.40196792 192.29547942  -5.74205021  -4.14257647  -1.85721645]
Checking original images
Original tensor(10.5070, grad_fn=<SelectBackward0>)
Perturbed tensor(10.5016, grad_fn=<SelectBackward0>)
5
[238.96248808  41.91960163  

KeyboardInterrupt: 